In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import matplotlib

In [10]:
%matplotlib widget

In [11]:
from overfit.trainers.overfit import OverfitTrainer
import matplotlib.pyplot as plt
from PIL import Image
from overfit.utils.img2vid import zigzag, display_video
import torchvision.transforms.functional as FT
from torchvision.models import resnet34, ResNet34_Weights
from torchvision.models import resnet50, ResNet50_Weights
import urllib
import mlflow
from pathlib import Path

In [12]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [13]:
input_image = Image.open("dog.jpg")
input_tensor = FT.to_tensor(input_image)
_, h, w = input_tensor.size()
input_video = zigzag(input_tensor, h // 3, w // 3)
# vid = display_video(input_video)
# vid.save("dog.mp4")
print(len(input_video))

69


In [14]:
def normalize_rgb(img):
    return FT.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_video = [normalize_rgb(frame).unsqueeze(0) for frame in input_video]

In [15]:
srcnet = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1).eval()
tgtnet_trainer = OverfitTrainer()
tgtnet_trainer.set(
    pretrained_classifier=srcnet,
    num_classes=1000,
    confidence=0.1,
    weight_decay=1,
    max_lr=0.1,
    momentum=0.9,
)

In [27]:
client = mlflow.MlflowClient(tracking_uri="http://localhost:5050")
experiment_id = "2"
run = client.create_run(experiment_id=experiment_id)

In [31]:
experiment = client.get_experiment(experiment_id)
print(experiment.artifact_location)
mlflow.pytorch.log_model(
    tgtnet_trainer.model, artifact_path=f"{experiment.artifact_location}/overfit", registered_model_name="overfit"
)
tgtnet_trainer.test(input_video, [258] * len(input_video))

file:///Users/dgcnz/development/research/overfitting-net/notebooks/../mlflow-artifacts


MissingConfigException: Yaml file '/Users/dgcnz/development/research/overfitting-net/notebooks/mlruns/0/meta.yaml' does not exist.